In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd


In [2]:
codes = pd.read_csv('London postcode districts.xlsx - PC DIST.csv')
codes.tail(1)

,Postcode district,Local Areas,Borough
267,WC2R,Somerset House,Westminster


In [3]:
def rightMove(i, Trans_type, website, df):
    s = Service("C:\\Users\\user\\Downloads\\Set ups\\chromedriver_win32\\chromedriver.exe")
    #s = Service("C:\\Users\\akint\\Downloads\\Set up files\\chromedriver.exe")
    driver = webdriver.Chrome(service= s)

    driver.get('https://www.rightmove.co.uk/')
    driver.maximize_window()
    time.sleep(2)

    time.sleep(1)
    search = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/main/div[1]/div/div/div/div/input')
    search.send_keys(postcode)

    time.sleep(2)
    driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/main/div[1]/div/div/div/button[1]').click()

    time.sleep(2)
    driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/div[1]/div/form/fieldset[2]/div[4]/button').click()

    Trans_type = []
    address = []
    types = []
    bedrooms = []
    bathrooms = []
    prices = []
    desc = []
    date_added = []
    agent_list = []
    property_url = []
    website = []
    
    df = pd.DataFrame()
    
    i = 1
    while True:
        time.sleep(2)
        print("{} {} {} {}".format('scraping page', i,'from', postcode ))
        address_list = driver.find_elements(by='class name', value='propertyCard-address')
        type_list = driver.find_elements(By.XPATH, "//div/div/div[4]/div[1]/div[2]/a/div/span[1]")
        bedroom_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[2]/a/div/span[3]')
        bathroom_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[2]/a/div/span[5]')
        price_list = driver.find_elements(By.XPATH, '//div/div/div[3]/div/a/div[1]')
        desc_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[3]/a/span/span')
        date_added_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[2]/div/span')
        agent_list_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[2]/div')
        property_url_list = driver.current_url
        Trans_type_list = Trans_type
        website_list = website
        for address_item, type_item, bedroom_item, bathroom_item, price_item, desc_item, date_added_item, agent_list_item in zip(address_list, type_list, bedroom_list, bathroom_list, price_list, desc_list, date_added_list, agent_list_list):
            address.append(address_item.text)
            types.append(type_item.text)
            bedrooms.append(bedroom_item.text)
            bathrooms.append(bathroom_item.text)
            prices.append(price_item.text)
            desc.append(desc_item.text)
            date_added.append(date_added_item.text)
            agent_list.append(agent_list_item.text)
            property_url.append(property_url_list)
            Trans_type.append(Trans_type_list)
            website.append(website_list)
        next_botton = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[3]/div/div/div/div[3]/button/span')
        last_botton_check = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[3]/div/div/div/div[3]/button')
        html = last_botton_check.get_attribute('outerHTML') 
        if 'disabled' in html:
            print('------------------------------- SCRAPING COMPLETED FOR ' + postcode)
            break
        next_botton.click()
        i += 1
        time.sleep(2)
        
    df1 = pd.DataFrame({'Location': postcode, 'Tansaction_Type': 'Sales','Property_Type':types, 'Address' :address, 'Bedrooms': bedrooms, 'Bathrooms':bathrooms, 'Price':prices, 'Description': desc, 'Listing_Date':date_added, 'Agent':agent_list, 'Listing_Source': 'RightMove', 'listing_URL':property_url})
    df1.insert(0, 'Unique_Id', [f'{postcode}{i+1:05d}' for i in range(len(df1))])
    print(df1.shape)
    
    df = pd.concat([df, df1], ignore_index=True)
    return df1



In [4]:
# create an empty DataFrame outside the function
df = pd.DataFrame()

# loop through postcodes
for postcode in codes.tail(2)['Postcode district']:
    # call the function and pass the empty DataFrame as an argument
    df1 = rightMove(postcode, 'Sales', 'RightMove', df)
    # append the df1 DataFrame to the empty DataFrame
    df = pd.concat([df, df1], ignore_index=True)
print('-------------------------------    ------------------------------- WEB SCRAPING COMPLETED')

scraping page 1 from WC2N
------------------------------- SCRAPING COMPLETED FOR WC2N
(18, 13)
scraping page 1 from WC2R
scraping page 2 from WC2R
scraping page 3 from WC2R
------------------------------- SCRAPING COMPLETED FOR WC2R
(51, 13)
-------------------------------    ------------------------------- WEB SCRAPING COMPLETED


In [5]:
df.shape

(69, 13)

In [7]:
df.head()

,Unique_Id,Location,Tansaction_Type,Property_Type,Address,Bedrooms,Bathrooms,Price,Description,Listing_Date,Agent,Listing_Source,listing_URL
0,WC2N00001,WC2N,Sales,Penthouse,"John Adam House, 17-19 John Adam Street, WC2N",2,2,"£1,050,000",An attractive penthouse apartment with two ter...,Added on 12/04/2023,Allow all cookies,RightMove,https://www.rightmove.co.uk/property-for-sale/...
1,WC2N00002,WC2N,Sales,Penthouse,Aria House,4,5,"£14,950,000",London House is delighted to bring to the mark...,Added on 17/03/2023,Added on 12/04/2023 by Chestertons Estate Agen...,RightMove,https://www.rightmove.co.uk/property-for-sale/...
2,WC2N00003,WC2N,Sales,Penthouse,"Northumberland Avenue, London, WC2N",4,4,"£14,950,000",An incredibly rare opportunity to purchase a p...,Added on 08/04/2022,"Added on 17/03/2023 by London House, London",RightMove,https://www.rightmove.co.uk/property-for-sale/...
3,WC2N00004,WC2N,Sales,Terraced,"12 Craven Street, Charing Cross, London, WC2N",5,5,"£3,800,000","Arranged over Lower Ground, Ground and Four Up...",Added on 17/04/2023,Added on 08/04/2022 by Sotheby's International...,RightMove,https://www.rightmove.co.uk/property-for-sale/...
4,WC2N00005,WC2N,Sales,Apartment,"Kipling House, Villiers Street, Strand WC2",2,2,"£2,499,950",Named after the famous author Rudyard Kipling ...,Added on 24/06/2022,"Added on 17/04/2023 by Allsop, Auction",RightMove,https://www.rightmove.co.uk/property-for-sale/...


In [3]:
def rightMove(i, Trans_type, website, dj):
    s = Service("C:\\Users\\user\\Downloads\\Set ups\\chromedriver_win32\\chromedriver.exe")
    #s = Service("C:\\Users\\akint\\Downloads\\Set up files\\chromedriver.exe")
    driver = webdriver.Chrome(service= s)

    driver.get('https://www.rightmove.co.uk/')
    driver.maximize_window()
    time.sleep(2)

    time.sleep(1)
    search = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/main/div[1]/div/div/div/div/input')
    search.send_keys(postcode)

    time.sleep(2)
    driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/main/div[1]/div/div/div/button[2]').click()
    

    time.sleep(2)
    driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/div[1]/div/form/fieldset[2]/div[4]/button').click()

    Trans_type = []
    address = []
    types = []
    bedrooms = []
    bathrooms = []
    prices = []
    desc = []
    date_added = []
    agent_list = []
    property_url = []
    website = []
    
    dj = pd.DataFrame()
    
    i = 1
    while True:
        time.sleep(2)
        print("{} {} {} {}".format('scraping page', i,'from', postcode ))
        address_list = driver.find_elements(by='class name', value='propertyCard-address')
        type_list = driver.find_elements(By.XPATH, "//div/div/div[4]/div[1]/div[2]/a/div/span[1]")
        bedroom_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[2]/a/div/span[3]')
        bathroom_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[2]/a/div/span[5]')
        price_list = driver.find_elements(By.XPATH, '//div/div/div[3]/div/a/div[1]')
        desc_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[3]/a/span/span')
        date_added_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[2]/div/span')
        agent_list_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[2]/div')
        property_url_list = driver.current_url
        Trans_type_list = Trans_type
        website_list = website
        for address_item, type_item, bedroom_item, bathroom_item, price_item, desc_item, date_added_item, agent_list_item in zip(address_list, type_list, bedroom_list, bathroom_list, price_list, desc_list, date_added_list, agent_list_list):
            address.append(address_item.text)
            types.append(type_item.text)
            bedrooms.append(bedroom_item.text)
            bathrooms.append(bathroom_item.text)
            prices.append(price_item.text)
            desc.append(desc_item.text)
            date_added.append(date_added_item.text)
            agent_list.append(agent_list_item.text)
            property_url.append(property_url_list)
            Trans_type.append(Trans_type_list)
            website.append(website_list)
        next_botton = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[3]/div/div/div/div[3]/button/span')
        last_botton_check = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[3]/div/div/div/div[3]/button')
        html = last_botton_check.get_attribute('outerHTML') 
        if 'disabled' in html:
            print('------------------------------- SCRAPING COMPLETED FOR ' + postcode)
            break
        next_botton.click()
        i += 1
        time.sleep(2)
        
    df1 = pd.DataFrame({'Location': postcode, 'Tansaction_Type': 'Sales','Property_Type':types, 'Address' :address, 'Bedrooms': bedrooms, 'Bathrooms':bathrooms, 'Price':prices, 'Description': desc, 'Listing_Date':date_added, 'Agent':agent_list, 'Listing_Source': 'RightMove', 'listing_URL':property_url})
    df1.insert(0, 'Unique_Id', [f'{postcode}{i+1:05d}' for i in range(len(df1))])
    print(df1.shape)
    
    dj = pd.concat([dj, df1], ignore_index=True)
    return df1



In [4]:
# create an empty DataFrame outside the function
dj = pd.DataFrame()

# loop through postcodes
for postcode in codes.tail(2)['Postcode district']:
    # call the function and pass the empty DataFrame as an argument
    df1 = rightMove(postcode, 'Rent', 'RightMove', dj)
    # append the df1 DataFrame to the empty DataFrame
    dj = pd.concat([dj, df1], ignore_index=True)
print('-------------------------------    ------------------------------- WEB SCRAPING COMPLETED')

scraping page 1 from WC2N
------------------------------- SCRAPING COMPLETED FOR WC2N
(0, 13)


KeyboardInterrupt: 